## Example 6.8: 2-Dimensional Newton-Raphson Warmup

As a warmup towards solving Exercise 6.2, implement a 2-Dimensional Newton-Raphson method:

$\vec{f} + \mathbf{J} \Delta \vec{x} = 0$ 

where: 

$\vec{f} = \left(\begin{array}{c} 
f_{1}  \\
f_{2}  \\
\end{array}\right)$, $\Delta \vec{x} = \left(\begin{array}{c} 
\Delta x_{1}  \\
\Delta x_{2}  \\
\end{array}\right)$, $\mathbf{J} = \left(\begin{array}{cccc} 
\frac{ \partial f_1 }{ \partial x_1 } & \frac{ \partial f_1 }{ \partial x_2 }\\
\frac{ \partial f_2 }{ \partial x_1 } & \frac{ \partial f_2 }{ \partial x_2 } 
\end{array}\right)$,

and solve via:  

$\mathbf{J} \Delta \vec{x} = -\vec{f}$.

using the ```linalg``` package.

You will need to: 

- Create a function that calculates the partial derivatives that enter the Jacobian, $\frac{ \partial f_i }{ \partial x_j }$. You may use the central-difference derivative.
- Create a function that calculates the actual Jacobian matrix, calling the aforementioned function.
- Create a function that returns the vector of functions $\vec{f}$.
- Use $\mathbf{J}$ and $-\vec{f}$ to solve the system iteratively for $\Delta \vec{x}$, adding it to $\vec{x}$, until a solution is reached, i.e. $|f_i| \sim 0$ within the required precision. 

(a) Check your Jacobian with the *linear* system:

$2 x + y - 13 = 0$

$x + y - 9 = 0$

(b) Use your code for the 2-D Newton Raphson to solve the linear system above. 

(c) Use your code to solve the nonlinear system:

$x^2 + y - 21 = 0$

$x + y^2 - 29 = 0$

Verify that your solutions correspond to the expected ones ($ x = 4, y = 5$).

(d) Use your code to solve the system:

$xe^y = 1$

$-x^2 + y = 1$

In [34]:
import numpy as np
from numpy import linalg
# modify from Chapter 4: 
# define a function for the central-difference derivative.
# x is now a NumPy array
# i is the component of the function (f_i) to be differentiated
# j is the component of the vector x to differentiate by (x_j)
def dfdt_CD_ND(funcvector, i, j, x, h): 
    """Calculates the central-difference partial derivative of a function func at vector x, in the j-th direction, with step size dx"""
    N = len(x) # get the length of the input array
    # increment only the j-th element by an amount dx
    dx = np.zeros(N) 
    dx[j] = h 
    return (funcvector(x+dx/2)[i] - funcvector(x-dx/2)[i])/h

In [42]:
# A higher-order function that calculates the Jacobian matrix for the N-dimensional vector of functions f_i:
# we will use a central-difference approximation to the derivatives, and a parameter h = 1E-5 for all directions 
def Jacobian(funcvector, x, h): # the input should be a NumPy array of functions, each of which is a function of N variables (x_i)
    """Calculate the Jacobian of the function vector as an NxN matrix using the central-difference approximation to the derivatives"""
    N = len(funcvector(x)) # get the number of dimensions
    output = np.zeros((N,N)) #  the output is an NxN NumPy array
    for i in range(N): # loop over the functions f_i
        for j in range(N): # loop over variables x_j:
            dfi_dxj = dfdt_CD_ND(funcvector, i,j,x,h)
            output[i][j] = dfi_dxj[0] # set this to the correct element of the Jacobian   
    return output 

In [44]:
# (a) CHECK Jacobian with linear functions:
def fi_check_linear(x):
    funcmatrix = np.array([[2*x[0] + x[1] - 13], [ x[0] + x[1] - 9 ]])
    return funcmatrix

# Perform check:
h=1E-5 # step length for derivatives
x0 = [1, 2] # point for evaluation derivatives (this will be irrelevant since the functions are linear)
print(Jacobian(fi_check_linear, x0, h))

[[2. 1.]
 [1. 1.]]


In [46]:
import numpy as np
from numpy import linalg 

# The N-D Newton-Raphson algorithm: 
# x0 is the initial guess, it should have the same dimensions as the number of variables! 
# Nmax is the number of evaluations
# prec is the required precision
# dx is the distance over which to take the central-difference derivative (not the same as the step size!)
def NewtonRaphsonND(func, x0, Nmax, prec, h): 
    """Function that implements the N-Dimensional Newton-Raphson algorithm for root finding"""
    # perform check that the number of dimensions is the same for the function and variables
    N = len(func(x0)) # get the number of dimensions
    if N != len(x0):
         raise Exception("The length of the function vector is not the same as the number of unknowns")
    n = 0 # the number of steps taken
    val = 1E99*np.ones(N) # the value of the equations, initialize to a large number
    roots = np.nan*np.ones(N) # initialize the roots to "not a number"
    for nn in range(Nmax): # loop runs up to the max number of evals, or up to the point where the precision is reached
        # get the values of the function at x0:
        minus_f = -func(x0)
        # check whether the required precision has been reached for *each* value:
        if np.all(np.abs(minus_f) < prec):
            n = nn # save the number of steps taken 
            print('Newton-Raphson Precision reached! Exiting')
            break # exit the loop nn
        # Get the Jacobian (J):
        J = Jacobian(func, x0, h)
        # calculate the step vector Dx using linear algebra (see Chapter 6, "NumPy's linalg Package" section)
        Dx = linalg.solve(J, minus_f).reshape(N) # turn this into the right shape (column vector)
        # update the guess and the value of the equation:
        x0 = np.add(x0, Dx)
    roots = x0
    return roots, n

In [49]:
# CHECK Jacobian with non-linear functions:
def fi_check_nonlinear(x):
    funcmatrix = np.array([[x[0]**2 + x[1] - 21], [ x[0] + x[1]**2 - 29 ]])
    return funcmatrix

# Perform check:
h=1E-5 # step length for derivatives
x0 = [1, 2] # point for evaluation derivatives
print(Jacobian(fi_check_nonlinear, x0, h))

[[2. 1.]
 [1. 4.]]


In [52]:
# (b) CHECK the linear function: 
Nmax = 1000
prec = 1E-6
h = 1E-5
x0 = np.array([0,0])
roots, niterations = NewtonRaphsonND(fi_check_linear, x0, Nmax, prec, h)
print('The roots of the linear equations are=', roots)

# (c) CHECK the non-linear function:
roots, niterations = NewtonRaphsonND(fi_check_nonlinear, x0, Nmax, prec, h)
print('The roots of the non-linear equations are=', roots)

Newton-Raphson Precision reached! Exiting
The roots of the linear equations are= [4. 5.]
Newton-Raphson Precision reached! Exiting
The roots of the non-linear equations are= [4.00000012 4.99999999]


In [54]:
# (d) solve the last nonlinear system:
def fi_check_nonlinear2(x):
    funcmatrix = np.array([[x[0] * np.exp(x[1]) - 1], [ -x[0]**2 + x[1] - 1]])
    return funcmatrix

# (c) CHECK the non-linear function:
roots, niterations = NewtonRaphsonND(fi_check_nonlinear2, x0, Nmax, prec, h)
print('The roots of the non-linear equations are=', roots)


Newton-Raphson Precision reached! Exiting
The roots of the non-linear equations are= [0.32993571 1.10885752]
